In [ ]:
import pandas as pd
import os

# Define file paths
DATA_PATH = 'C:\\Users\\edinocencio\\DataClenupCmsCrm\\DataSources\\'
FINALIZED_PATH = os.path.join(DATA_PATH, 'finalizados')
MASTER_FILE = os.path.join(DATA_PATH, 'PMD-CMS.xlsx')
INVOICE_FILE = os.path.join(DATA_PATH, 'GetInvoice.xlsx')
RECENT_CONTRACTS_FILE = os.path.join(FINALIZED_PATH, 'recent_contracts.xlsx')
UNIQUE_PARTNERS_FILE = os.path.join(FINALIZED_PATH, 'unique_partner_ids.xlsx')
SHEET_NAME = 'after synch'

# Function to load Excel data
def load_excel(file_path, sheet_name=None, usecols=None):
    return pd.read_excel(file_path, sheet_name=sheet_name, usecols=usecols)


# Function to identify most recent contracts for duplicate PartnerIDs
def get_most_recent_contracts(df, group_by_column, date_column):
    # Identify duplicate PartnerIDs
    duplicated_ids = df[df.duplicated(subset=group_by_column, keep=False)][group_by_column].unique()

    # Filter DataFrame for duplicated PartnerIDs
    duplicated_df = df[df[group_by_column].isin(duplicated_ids)]

    # Get the most recent contract for each duplicate PartnerID
    recent_contracts = duplicated_df.loc[
        duplicated_df.groupby(group_by_column)[date_column].idxmax()
    ]

    return recent_contracts, duplicated_ids

# Function to save DataFrame to Excel
def save_to_excel(df, file_path):
    df.to_excel(file_path, index=False)


# Load master data
df_master = load_excel(MASTER_FILE, sheet_name=SHEET_NAME)

# Set headers and select necessary columns
HEADERS = df_master.iloc[0, :].tolist()
df_master.columns = HEADERS
df_master = df_master.loc[1:,['PartnerID', 'Branch','ContractNumber']]

# Create a new 'CONTRACT' column
df_master['CONTRACT'] = df_master['Branch'].astype(str) + "-" + df_master['ContractNumber'].astype(str)

# Load invoice data
df_getinvoice = load_excel(INVOICE_FILE, usecols=['B+C','Acceptance Date'])

# Merge master data with invoice data
df_master = df_master.merge(df_getinvoice,left_on='CONTRACT', right_on='B+C', how='left')

# Convert the 'Acceptance Date' column to datetime
df_master['Acceptance Date'] = pd.to_datetime(df_master['Acceptance Date'])

# Get most recent contracts for duplicated PartnerIDs
df_recent_contracts, duplicated_partner_ids = get_most_recent_contracts(
    df_master, group_by_column='PartnerID', 
    date_column='Acceptance Date'
)

# Criar DataFrame com PartnerIDs únicos
df_unique = df_master[~df_master['PartnerID'].isin(duplicated_partner_ids)]

# Salvar resultados em arquivos separados
save_to_excel(df_recent_contracts, RECENT_CONTRACTS_FILE)
save_to_excel(df_unique, UNIQUE_PARTNERS_FILE)

print("Os contratos mais recentes foram salvos em 'recent_contracts.xlsx'.")
print("Os PartnerIDs únicos foram salvos em 'unique_partner_ids.xlsx'.")
